In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# IMPORTANT: Update this path to where you uploaded the 'Google-Colab' folder!
import os
project_path = '/content/drive/MyDrive/Google-Colab'
os.chdir(project_path)

# Verify the current working directory and list files to confirm
print(f"Current working directory: {os.getcwd()}")
print("Files in current directory:")
for item in os.listdir('.'):
    print(item)

In [ ]:
# ========================================================================
# 🔧 CONFIGURE YOUR EXPERIMENT HERE
# ========================================================================
DATASET = 'amazon'  # Options: 'yelp', 'amazon', 'gowalla', 'movielens'

# Grid Search Configuration for Hard Negative Sampling
GRID_SEARCH_ENABLED = True  # Set to False for single experiment
HARD_NEG_SAMPLES_K = [3, 5, 7]  # Number of hard negatives to test
CONTRASTIVE_WEIGHTS = [0.2, 0.1, 0.01, 0]  # Contrastive loss weights (λ)
GRID_SEARCH_EPOCHS = 25  # Epochs per grid search experiment

# Single experiment configuration (used when GRID_SEARCH_ENABLED = False)
SINGLE_K = 5
SINGLE_LAMBDA = 0.1
SINGLE_EPOCHS = 150

# Google Drive results path
DRIVE_RESULTS_PATH = '/content/drive/MyDrive/HardGNN_Results'  # Results will be saved here
# ========================================================================

In [ ]:
# ========================================================================
# CELL 1: Environment Setup - PRIORITIZING COLAB DEFAULTS for TF/NumPy
# ========================================================================
import subprocess
import sys
import os
import site

def install_missing_dependencies():
    """Install/upgrade non-ML core dependencies. NumPy and TensorFlow should be Colab's defaults."""
    print("Installing/upgrading non-ML core dependencies for HardGNN...")
    print(f"Detected Python version: {sys.version}")

    # We rely on Colab's pre-installed versions.
    dependencies = [
        "matplotlib>=3.5.0",
        "scipy>=1.12.0",
        "protobuf>=3.19.0,<4.25.0",
        "pandas>=1.3.0",
        "scikit-learn>=1.0.0"
    ]

    print("Attempting to install/upgrade non-ML core dependencies to user site...")
    for dep in dependencies:
        print(f"Processing {dep}...")
        try:
            command = [sys.executable, "-m", "pip", "install", "--no-cache-dir", "--upgrade", "--user", dep]
            print(f"   Executing: {' '.join(command)}")
            result = subprocess.run(command,
                                  check=True, capture_output=True, text=True, timeout=180)
        except subprocess.CalledProcessError as e:
            print(f"Warning: Could not process {dep}. Pip error: {e}")
        except subprocess.TimeoutExpired as e:
            print(f"Timeout: Processing of {dep} took too long.")
        except Exception as e:
            print(f"An unexpected error occurred processing {dep}: {e}")

    print("Non-ML core dependency processing complete.")
    try:
        # Ensure user site packages are in path
        if hasattr(site, 'USER_SITE') and site.USER_SITE and site.USER_SITE not in sys.path:
            sys.path.insert(0, site.USER_SITE)
        # For Colab/Linux, also consider adding local/bin to PATH if it exists for any pip installed CLIs
        local_bin_path = os.path.expanduser("~/.local/bin")
        if os.path.isdir(local_bin_path) and local_bin_path not in os.environ['PATH']:
            os.environ['PATH'] = local_bin_path + os.pathsep + os.environ['PATH']
    except Exception as e:
        print(f"Could not update sys.path/PATH for user site: {e}")

def setup_tensorflow_compatibility(tf_module, numpy_module):
    print(f"Using TensorFlow version: {tf_module.__version__ if tf_module else 'N/A'}")
    print(f"Using NumPy version: {numpy_module.__version__ if numpy_module else 'N/A'}")

    if not tf_module or not numpy_module:
        return False

    # Informational checks about loaded versions
    if numpy_module.__version__.startswith("2."):
        print(f"NumPy version is {numpy_module.__version__}. Colab's TensorFlow ({tf_module.__version__}) should be compatible (e.g., >=2.16 or specially built)." )
    elif numpy_module.__version__.startswith("1."):
        print(f"NumPy version is {numpy_module.__version__}. Colab's TensorFlow ({tf_module.__version__}) should be compatible (e.g., <=2.15 or specially built)." )
    else:
        print(f"Unknown NumPy version pattern: {numpy_module.__version__}")

    try:
        import ml_dtypes
        print(f"ml_dtypes version found: {ml_dtypes.__version__} (from {ml_dtypes.__file__})")
        if numpy_module.__version__.startswith("2.") and not ml_dtypes.__version__.startswith(("0.4", "0.5")):
            print(f"WARNING: ml_dtypes version ({ml_dtypes.__version__}) might not be ideal for NumPy 2.x (expected 0.4.x or 0.5.x). Check for runtime issues.")
        elif numpy_module.__version__.startswith("1.") and ml_dtypes.__version__.startswith(("0.4", "0.5")):
             print(f"WARNING: ml_dtypes version ({ml_dtypes.__version__}) might not be ideal for NumPy 1.x (expected <0.4.x). Check for runtime issues.")
    except ImportError:
        print("ml_dtypes not explicitly found or importable by script. TensorFlow might bundle it or manage it internally.")
    except Exception as e:
        print(f"Error during ml_dtypes check: {e}")

    try:
        tf_module.compat.v1.disable_eager_execution()
        tf_module.compat.v1.disable_v2_behavior()
        gpus = tf_module.config.list_physical_devices('GPU')
        if gpus:
            print(f"GPU acceleration available: {len(gpus)} GPU(s) detected")
            for gpu_device in gpus:
                print(f"   - {gpu_device}")
                try:
                    tf_module.config.experimental.set_memory_growth(gpu_device, True)
                    print(f"GPU memory growth configured for {gpu_device}")
                except RuntimeError as e:
                    print(f"Could not configure GPU memory growth for {gpu_device}: {e}")
        else:
            print("No GPU detected, will use CPU.")
        return True
    except AttributeError as e:
        return False
    except Exception as e:
        return False

def verify_colab_environment(tf_module, numpy_module):
    import sys
    import psutil
    import platform

    print(f"Python: {sys.version}")
    print(f"sys.path (first few entries): {str(sys.path[:5])}")
    print(f"Platform: {platform.platform()}")
    print(f"Architecture: {platform.machine()}")

    if numpy_module:
        print(f"NumPy Version (loaded): {numpy_module.__version__}")
        print(f"NumPy Path: {numpy_module.__file__}")
    else:
        print("NumPy Version (loaded): NOT LOADED")

    if tf_module:
        print(f"TensorFlow Version (loaded): {tf_module.__version__}")
        print(f"TensorFlow Path: {tf_module.__file__}")
    else:
        print("TensorFlow Version (loaded): NOT LOADED")

    try:
        import ml_dtypes
        print(f"ml_dtypes Version (loaded): {ml_dtypes.__version__} from {ml_dtypes.__file__}")
    except Exception:
        print(f"ml_dtypes: Not found by script or error during import check (may be internal to TF).")

    try:
        import tensorflow_metadata
        print(f"tensorflow-metadata Version (loaded): {tensorflow_metadata.__version__} from {tensorflow_metadata.__file__}")
    except Exception:
        print(f"tensorflow-metadata: Not found by script or error (may not be needed or part of default TF).")

    memory = psutil.virtual_memory()
    memory_gb = memory.total / (1024**3)
    print(f"Available RAM: {memory_gb:.1f} GB")
    disk = psutil.disk_usage('/')
    disk_gb = disk.free / (1024**3)
    print(f"Available disk space: {disk_gb:.1f} GB")
    return True

# --- Main Execution Flow ---
print("=" * 60)
print("HardGNN Setup for Google Colab Pro+ (PRIORITIZING COLAB DEFAULTS for TF/NumPy)")
print("=" * 60)

# 1. Install/Upgrade other dependencies (non TF/NumPy)
install_missing_dependencies()

# 2. Import Colab's default NumPy and TensorFlow
numpy_to_use = None
tensorflow_to_use = None

try:
    import numpy
    numpy_to_use = numpy
except Exception as e:
    print("This is a critical failure. Further steps will likely fail.")

try:
    import tensorflow
    tensorflow_to_use = tensorflow
except Exception as e:
    print(f"This could be due to an underlying issue with its dependencies (like the loaded NumPy version) or Colab environment configuration.")

if not numpy_to_use or not tensorflow_to_use:
    print("CRITICAL FAILURE: Could not import Colab's default NumPy or TensorFlow. Environment setup will likely be incomplete or fail.")

# 3. Setup TensorFlow compatibility using the imported Colab modules
setup_successful = False # Default to False
if numpy_to_use and tensorflow_to_use:
    setup_successful = setup_tensorflow_compatibility(tf_module=tensorflow_to_use, numpy_module=numpy_to_use)
else:
    print("Skipping TensorFlow compatibility setup as core modules (NumPy/TensorFlow) failed to load.")

# 4. Verify environment using the imported Colab modules
verify_colab_environment(tf_module=tensorflow_to_use, numpy_module=numpy_to_use)

if not setup_successful:
    # Custom error message based on whether TF/NumPy even loaded
    if not numpy_to_use or not tensorflow_to_use:
        raise RuntimeError("TensorFlow/NumPy native import failed. Cannot configure environment.")
    else:
        raise RuntimeError("TensorFlow setup failed using Colab's default versions. There might be an incompatibility within the pre-built Colab environment, or the TF1 compatibility layer cannot be applied to the loaded versions.")

print("Environment setup attempt complete using Colab's default TF/NumPy (or best effort)!")

In [ ]:
# ========================================================================
# CELL 2: Dataset Configuration and Module Import
# ========================================================================
if tensorflow_to_use:
    tf = tensorflow_to_use
else:
    import tensorflow as tf
    print("Warning: tensorflow_to_use was not set from Cell 1. Attempted direct import of tensorflow as tf.")

# Core Python Standard Library imports
import os
import random
import pickle
import json
from datetime import datetime
import time
import gc
import traceback
import queue
import threading
import multiprocessing as mp

# Third-party library imports
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import pandas as pd

# Import our modules
from Params import args
import Utils.TimeLogger as logger
from Utils.TimeLogger import log
from DataHandler import DataHandler

# Import the HardGNN model
print("Importing HardGNN model...")
try:
    from HardGNN_model import Recommender
    print("Successfully imported HardGNN model")
except ImportError as e:
    print(f"Failed to import HardGNN model: {e}")
    sys.exit(1)

def configure_dataset(dataset_name, hard_neg_k=5, contrastive_weight=0.1):
    """Configure parameters based on validated configurations for each dataset"""

    # Set base dataset
    args.data = dataset_name.lower()

    # Dataset-specific validated configurations
    if dataset_name.lower() == 'yelp':
        args.lr = 1e-3
        args.reg = 1e-2
        args.temp = 0.1
        args.ssl_reg = 1e-7
        args.epoch = 150
        args.batch = 512
        args.sslNum = 40
        args.graphNum = 12
        args.gnn_layer = 3
        args.att_layer = 2
        args.testSize = 1000
        args.ssldim = 32
        args.sampNum = 40

    elif dataset_name.lower() == 'amazon':
        args.lr = 1e-3
        args.reg = 1e-2
        args.temp = 0.1
        args.ssl_reg = 1e-6
        args.epoch = 150
        args.batch = 512
        args.sslNum = 80
        args.graphNum = 5
        args.pred_num = 0
        args.gnn_layer = 3
        args.att_layer = 4
        args.testSize = 1000
        args.keepRate = 0.5
        args.sampNum = 40
        args.pos_length = 200

    elif dataset_name.lower() == 'gowalla':
        args.lr = 2e-3
        args.reg = 1e-2
        args.temp = 0.1
        args.ssl_reg = 1e-6
        args.epoch = 150
        args.batch = 512
        args.graphNum = 3
        args.gnn_layer = 2
        args.att_layer = 1
        args.testSize = 1000
        args.sampNum = 40

    elif dataset_name.lower() == 'movielens':
        args.lr = 1e-3
        args.reg = 1e-2
        args.ssl_reg = 1e-6
        args.epoch = 150
        args.batch = 512
        args.sampNum = 40
        args.sslNum = 90
        args.graphNum = 6
        args.gnn_layer = 2
        args.att_layer = 3
        args.testSize = 1000
        args.ssldim = 48
        args.keepRate = 0.5
        args.pos_length = 200
        args.leaky = 0.5

    else:
        print("Available datasets: yelp, amazon, gowalla, movielens")

    # Handle edge cases for hard negative sampling and contrastive loss
    if hard_neg_k == 0:
        # K=0: Disable hard negative sampling entirely
        args.use_hard_neg = False
        args.hard_neg_top_k = 0
    else:
        # K>0: Enable hard negative sampling
        args.use_hard_neg = True
        args.hard_neg_top_k = hard_neg_k

    # Set contrastive weight (λ=0 is handled in model during loss computation)
    args.contrastive_weight = contrastive_weight
    # Note: τ (temperature) is already set in args.temp = 0.1

    # Performance optimization for hard negative sampling
    args.cache_refresh_steps = 25  # Refresh embeddings every N training steps

    # Enable AMP to be handled within the model
    args.enable_amp = True # This will be read by HardGNN_model.py

    args.tstEpoch = 3  # Test every 3 epochs (can be adjusted if needed for full runs)

    # Set save path
    args.save_path = f'hardgnn_{dataset_name.lower()}_k{hard_neg_k}_lambda{contrastive_weight}'

    return args

# Configure the dataset
if GRID_SEARCH_ENABLED:
    # Use first combination for initial setup
    configure_dataset(DATASET, HARD_NEG_SAMPLES_K[0], CONTRASTIVE_WEIGHTS[0])
    print(f"   K values: {HARD_NEG_SAMPLES_K}")
    print(f"   λ values: {CONTRASTIVE_WEIGHTS}")
    print(f"   Results will be saved to: {DRIVE_RESULTS_PATH}")
else:
    configure_dataset(DATASET, SINGLE_K, SINGLE_LAMBDA)
    args.epoch = SINGLE_EPOCHS

# Initialize grid search results storage
if GRID_SEARCH_ENABLED:
    grid_search_results = []
    # Create results directory in Google Drive
    os.makedirs(DRIVE_RESULTS_PATH, exist_ok=True)

if GRID_SEARCH_ENABLED:
    print(f"  Mode: Grid Search")
    print(f"  K range: {HARD_NEG_SAMPLES_K}")
    print(f"  λ range: {CONTRASTIVE_WEIGHTS}")
else:
    print(f"  Mode: Single Experiment")
    print(f"  Hard Negatives (K): {args.hard_neg_top_k}")
    print(f"  Contrastive Weight (λ): {args.contrastive_weight}")

In [ ]:
# ========================================================================
# CELL 3: Load Dataset
# ========================================================================

# Initialize and load data
logger.saveDefault = True
log(f'Starting {DATASET} data loading...')

handler = DataHandler()
handler.LoadData()

log(f'{DATASET} data loaded successfully')
print(f"{DATASET.upper()} Dataset Statistics:")
print(f"  Users: {args.user:,}")
print(f"  Items: {args.item:,}")
print(f"  Training interactions: {handler.trnMat.nnz:,}")
print(f"  Test users: {len(handler.tstUsrs):,}")
print(f"  Time-based graphs: {len(handler.subMat)}")

In [ ]:
# ========================================================================
# CELL 4: Validate Contrastive Loss Component
# ========================================================================

print(f" Validating Hard Negative Sampling on {DATASET}...")
print(f" Testing with τ={args.temp}, K={args.hard_neg_top_k}, λ={args.contrastive_weight}")

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.compat.v1.set_random_seed(42)

# Initialize TensorFlow session with GPU config
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True

with tf.compat.v1.Session(config=config) as sess:
    # Initialize HardGNN model
    model = Recommender(sess, handler)
    model.prepareModel()

    # Initialize variables
    init = tf.compat.v1.global_variables_initializer()
    sess.run(init)
    log(' Model initialized (random weights)')

    # Test contrastive loss on a small batch
    test_users = handler.tstUsrs[:32]  # Small batch for validation

    try:
        # Sample batch with hard negatives
        uLocs, iLocs, sequence, mask, uLocs_seq = model.sampleTrainBatch(
            test_users, handler.trnMat, handler.timeMat, train_sample_num=10
        )

        # Sample SSL batch
        suLocs, siLocs, suLocs_seq = model.sampleSslBatch(test_users, handler.subMat, False)

        # Prepare feed dict
        feed_dict = {
            model.uids: uLocs,
            model.iids: iLocs,
            model.sequence: sequence,
            model.mask: mask,
            model.is_train: False,
            model.uLocs_seq: uLocs_seq,
            model.keepRate: 1.0
        }

        for k in range(args.graphNum):
            feed_dict[model.suids[k]] = suLocs[k]
            feed_dict[model.siids[k]] = siLocs[k]
            feed_dict[model.suLocs_seq[k]] = suLocs_seq[k]

        # Run forward pass
        if hasattr(model, 'contrastive_loss'):
            results = sess.run([
                model.contrastive_loss,
                model.preLoss,
                model.posPred,
                model.negPred
            ], feed_dict=feed_dict)

            contrastive_loss, pre_loss, pos_pred, neg_pred = results

            print(f"🎯 HARD NEGATIVE SAMPLING VALIDATION - {DATASET.upper()}")
            print(f"📊 Metrics:")
            print(f"  Contrastive Loss: {contrastive_loss:.6f}")
            print(f"  Supervised Loss: {pre_loss:.6f}")
            print(f"  Positive Predictions: {np.mean(pos_pred):.4f} ± {np.std(pos_pred):.4f}")
            print(f"  Negative Predictions: {np.mean(neg_pred):.4f} ± {np.std(neg_pred):.4f}")
            print(f"  Prediction Gap: {np.mean(pos_pred) - np.mean(neg_pred):.4f}")

            if np.mean(pos_pred) > np.mean(neg_pred):
                print("  Positive predictions > Negative predictions")
            else:
                print("  Negative predictions >= Positive predictions")

            if contrastive_loss > 0 and not np.isnan(contrastive_loss):
                print("  Hard negative sampling working correctly")
            else:
                print("  Issue with hard negative sampling")

            print(f"\n Validation Complete - Ready for {DATASET.upper()} Training!")
        else:
            print(" Hard negative sampling not available")

    except Exception as e:
        print(f" Validation error: {e}")
        print("Proceeding with training...")


In [ ]:
# ========================================================================
# CELL 5: Grid Search Training or Single Experiment
# ========================================================================

def save_experiment_result_to_drive(result, dataset_name, experiment_num=None):
    """Save individual experiment result to Google Drive"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    if experiment_num is not None:
        filename = f"hardgnn_{dataset_name}_exp{experiment_num:02d}_k{result['k_value']}_lambda{result['lambda_value']}_{timestamp}.json"
    else:
        filename = f"hardgnn_{dataset_name}_single_k{result['k_value']}_lambda{result['lambda_value']}_{timestamp}.json"

    filepath = os.path.join(DRIVE_RESULTS_PATH, filename)

    # Save detailed result
    with open(filepath, 'w') as f:
        json.dump(result, f, indent=2, default=str)

    print(f" Result saved to Drive: {filename}")
    return filepath

def save_grid_search_summary_to_drive(all_results, dataset_name):
    """Save complete grid search summary to Google Drive"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Save JSON summary
    json_filename = f"hardgnn_{dataset_name}_grid_search_summary_{timestamp}.json"
    json_filepath = os.path.join(DRIVE_RESULTS_PATH, json_filename)

    with open(json_filepath, 'w') as f:
        json.dump({
            'dataset': dataset_name,
            'grid_search_config': {
                'k_values': HARD_NEG_SAMPLES_K,
                'lambda_values': CONTRASTIVE_WEIGHTS,
                'epochs_per_experiment': GRID_SEARCH_EPOCHS
            },
            'results': all_results,
            'timestamp': timestamp
        }, f, indent=2, default=str)

    # Save CSV summary
    csv_filename = f"hardgnn_{dataset_name}_grid_search_summary_{timestamp}.csv"
    csv_filepath = os.path.join(DRIVE_RESULTS_PATH, csv_filename)

    df = pd.DataFrame(all_results)
    df.to_csv(csv_filepath, index=False)

    print(f" Grid search summary saved to Drive:")
    print(f"   JSON: {json_filename}")
    print(f"   CSV: {csv_filename}")

    return json_filepath, csv_filepath

def run_single_experiment(k_value, lambda_value, epochs, experiment_num=None, total_experiments=None):
    """Run a single experiment with given hyperparameters - GPU optimized"""

    # Reset TensorFlow graph for fresh start - THIS MUST BE FIRST
    tf.compat.v1.reset_default_graph()

    # Also reset NNLayers_tf2 global parameter tracking
    from Utils import NNLayers_tf2
    NNLayers_tf2.reset_nn_params()

    # Configure for this experiment (this will set args.enable_amp = True)
    configure_dataset(DATASET, k_value, lambda_value)
    args.epoch = epochs

    # Create experiment identifier
    exp_id = f"K{k_value}_λ{lambda_value}"
    if experiment_num is not None:
        exp_header = f"Experiment {experiment_num}/{total_experiments}: {exp_id}"
    else:
        exp_header = f"Experiment: {exp_id}"

    print(f"{exp_header}")
    print(f"Configuration: K={k_value}, λ={lambda_value}, Epochs={epochs}")

    # Determine experiment type for edge case documentation
    if k_value == 0 and lambda_value == 0:
        experiment_type = "Pure SelfGNN (no hard negatives, no contrastive loss)"
    elif k_value == 0 and lambda_value > 0:
        experiment_type = "SelfGNN + Contrastive only (no hard negatives)"
    elif k_value > 0 and lambda_value == 0:
        experiment_type = "SelfGNN + Hard negatives only (no contrastive loss)"
    else:
        experiment_type = "Full HardGNN (hard negatives + contrastive loss)"

    print(f"   Experiment Type: {experiment_type}")
    print(f"   Hard Negative Sampling: {'Disabled' if k_value == 0 else 'Enabled'}")
    print(f"   Contrastive Loss: {'Disabled (λ=0)' if lambda_value == 0 else f'Enabled (λ={lambda_value})'}")

    # GPU-optimized TensorFlow session configuration
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.95  # Use most of GPU memory

    if tf.config.list_physical_devices('GPU'):
        config.graph_options.optimizer_options.global_jit_level = tf.compat.v1.OptimizerOptions.ON_1
        # config.gpu_options.experimental.enable_async_io = True # Ensure this is commented out or removed
        log_message_parts = [" GPU optimization enabled: XLA JIT", "95% memory allocation"]
        # AMP status is now logged by the model, but we can mention it's attempted.
        log_message_parts.append("AMP attempted in model")
        print(f"{', '.join(log_message_parts)}. Async I/O removed.")

    # Enable intra-op and inter-op parallelism for CPU efficiency
    config.intra_op_parallelism_threads = mp.cpu_count()
    config.inter_op_parallelism_threads = mp.cpu_count() // 2
    config.use_per_session_threads = True

    experiment_start_time = datetime.now()

    sess = None # Initialize sess to None for the finally block
    try:
        with tf.compat.v1.Session(config=config) as sess:
            # Initialize model
            model = Recommender(sess, handler)
            model.prepareModel()

            # Initialize variables
            init = tf.compat.v1.global_variables_initializer()
            sess.run(init)

            # Training tracking
            best_hr = 0.0
            best_ndcg = 0.0
            best_epoch = 0
            best_results = {}
            final_hr = 0.0
            final_ndcg = 0.0
            final_epoch = epochs
            final_results = {}
            epoch_results = []

            # Pipeline optimization: Pre-compute user batches for parallel processing
            print(f" Optimizing data pipeline for parallel processing...")
            num_users = args.user
            batch_size = args.batch
            user_batches = []

            # Pre-generate all user ID permutations for faster training
            for ep in range(epochs):
                sfIds = np.random.permutation(num_users)[:args.trnNum]
                epoch_batches = []
                steps = int(np.ceil(len(sfIds) / batch_size))
                for i in range(steps):
                    st = i * batch_size
                    ed = min((i+1) * batch_size, len(sfIds))
                    epoch_batches.append(sfIds[st:ed])
                user_batches.append(epoch_batches)

            # Training loop with pipeline optimization
            for ep in range(epochs):
                test = (ep % args.tstEpoch == 0)

                if ep % 5 == 0 or test:
                    print(f" Epoch {ep+1}/{epochs} (K={k_value}, λ={lambda_value}) - GPU Pipeline Active")

                # GPU-optimized training with pre-computed batches
                train_results = train_epoch_optimized(model, user_batches[ep], sess)

                # Test if it's a test epoch
                if test:
                    test_results = model.testEpoch()
                    hr = test_results['HR']
                    ndcg = test_results['NDCG']

                    # Track best results
                    if ndcg > best_ndcg:
                        best_ndcg = ndcg
                        best_hr = hr
                        best_epoch = ep + 1  # Convert to 1-indexed
                        best_results = test_results.copy()
                        print(f" New best: HR={hr:.4f}, NDCG={ndcg:.4f} (Epoch {ep+1})")

                    # Store epoch results
                    epoch_results.append({
                        'epoch': ep+1,
                        'hr': hr,
                        'ndcg': ndcg,
                        'train_loss': train_results.get('Loss', 0),
                        'pre_loss': train_results.get('preLoss', 0),
                        'contrastive_loss': train_results.get('contrastiveLoss', 0)
                    })

            # Final evaluation
            final_test_results = model.testEpoch()
            final_hr = final_test_results['HR']
            final_ndcg = final_test_results['NDCG']
            final_results = final_test_results.copy()
            experiment_duration = (datetime.now() - experiment_start_time).total_seconds()

            # ========================================================================
            # COMPREHENSIVE RESULTS PRINTING AND LOGGING
            # ========================================================================
            print("\n" + "="*80)
            print(f" EXPERIMENT RESULTS: {exp_id}")
            print("="*80)
            print(f" Experiment Type: {experiment_type}")
            print(f"  Configuration:")
            print(f"   • K (Hard negatives): {k_value}")
            print(f"   • λ (Contrastive weight): {lambda_value}")
            print(f"   • Epochs: {epochs}")
            print(f"   • Duration: {experiment_duration/60:.1f} minutes")
            print(f"   • GPU Efficiency: {epochs*len(user_batches[0])/(experiment_duration/60):.1f} batches/min")
            print()

            print(f" BEST PERFORMANCE:")
            print(f"   • Best HR: {best_hr:.4f}")
            print(f"   • Best NDCG: {best_ndcg:.4f}")
            print(f"   • Best Epoch: {best_epoch}")
            if 'HR5' in best_results:
                print(f"   • Best HR@5: {best_results['HR5']:.4f}")
                print(f"   • Best NDCG@5: {best_results['NDCG5']:.4f}")
            if 'HR20' in best_results:
                print(f"   • Best HR@20: {best_results['HR20']:.4f}")
                print(f"   • Best NDCG@20: {best_results['NDCG20']:.4f}")
            print()

            print(f" FINAL PERFORMANCE (Epoch {final_epoch}):")
            print(f"   • Final HR: {final_hr:.4f}")
            print(f"   • Final NDCG: {final_ndcg:.4f}")
            if 'HR5' in final_results:
                print(f"   • Final HR@5: {final_results['HR5']:.4f}")
                print(f"   • Final NDCG@5: {final_results['NDCG5']:.4f}")
            if 'HR20' in final_results:
                print(f"   • Final HR@20: {final_results['HR20']:.4f}")
                print(f"   • Final NDCG@20: {final_results['NDCG20']:.4f}")
            print()

            # Performance comparison
            hr_improvement = ((final_hr - best_hr) / best_hr * 100) if best_hr > 0 else 0
            ndcg_improvement = ((final_ndcg - best_ndcg) / best_ndcg * 100) if best_ndcg > 0 else 0
            print(f"📈 PERFORMANCE ANALYSIS:")
            print(f"   • HR improvement (final vs best): {hr_improvement:+.2f}%")
            print(f"   • NDCG improvement (final vs best): {ndcg_improvement:+.2f}%")

            if best_epoch < final_epoch:
                print(f"   • Best performance at epoch {best_epoch}, final at epoch {final_epoch}")
                print(f"   • Potential overfitting detected ({final_epoch - best_epoch} epochs after best)")
            else:
                print(f"   • Best performance maintained until final epoch")

            print("="*80)

            # Comprehensive experiment result for logging
            experiment_result = {
                'dataset': DATASET,
                'experiment_type': experiment_type,
                'k_value': k_value,
                'lambda_value': lambda_value,
                'epochs': epochs,
                'duration_minutes': experiment_duration/60,
                'gpu_efficiency_batches_per_min': epochs*len(user_batches[0])/(experiment_duration/60) if experiment_duration > 0 else 0,

                # Best performance metrics
                'best_hr': best_hr,
                'best_ndcg': best_ndcg,
                'best_epoch': best_epoch,
                'best_hr5': best_results.get('HR5', None),
                'best_ndcg5': best_results.get('NDCG5', None),
                'best_hr20': best_results.get('HR20', None),
                'best_ndcg20': best_results.get('NDCG20', None),

                # Final performance metrics
                'final_hr': final_hr,
                'final_ndcg': final_ndcg,
                'final_epoch': final_epoch,
                'final_hr5': final_results.get('HR5', None),
                'final_ndcg5': final_results.get('NDCG5', None),
                'final_hr20': final_results.get('HR20', None),
                'final_ndcg20': final_results.get('NDCG20', None),

                # Performance analysis
                'hr_improvement_percent': hr_improvement,
                'ndcg_improvement_percent': ndcg_improvement,
                'potential_overfitting': best_epoch < final_epoch,
                'epochs_after_best': max(0, final_epoch - best_epoch),

                # Technical details
                'hard_negatives_enabled': k_value > 0,
                'contrastive_loss_enabled': lambda_value > 0,
                'timestamp': experiment_start_time.isoformat(),

                # Detailed epoch-by-epoch results
                'epoch_details': epoch_results,

                # Full results objects for reference
                'best_results_full': best_results,
                'final_results_full': final_results
            }

            print(f" Experiment completed successfully!")
            print(f" Saving detailed results to Google Drive...")

            # Save individual result to Google Drive
            save_experiment_result_to_drive(experiment_result, DATASET, experiment_num)

            return experiment_result

    except Exception as e:
        print(f"Experiment {exp_id} failed: {e}")
        error_trace = traceback.format_exc()
        print(f"Traceback:\\n{error_trace}")
        # Log partial or error state if necessary
        return {
            "experiment_id": exp_id,
            "k_value": k_value,
            "lambda_value": lambda_value,
            "epochs_configured": epochs,
            "status": "Failed",
            "error_message": str(e),
            "error_traceback": error_trace,
            "duration_seconds": (datetime.now() - experiment_start_time).total_seconds(),
            "experiment_type": experiment_type,
        }
    finally:
        if sess is not None:
            sess.close()
        # Force garbage collection to free up memory, especially GPU memory
        gc.collect()

def train_epoch_optimized(model, batch_list, sess):
    """GPU-optimized training epoch with pipeline parallelization"""
    epochLoss, epochPreLoss = [0] * 2
    epochContrastiveLoss = 0
    sample_num_list = [40]
    steps = len(batch_list)

    # Pre-allocate arrays for better memory efficiency
    batch_queue = queue.Queue(maxsize=3)  # Small buffer for pipeline overlap

    def data_loader_worker():
        """Background thread for data loading pipeline"""
        for s in range(len(sample_num_list)):
            for i, batIds in enumerate(batch_list):
                try:
                    # Pre-compute batch data
                    uLocs, iLocs, sequence, mask, uLocs_seq = model.sampleTrainBatch(
                        batIds, model.handler.trnMat, model.handler.timeMat, sample_num_list[s]
                    )
                    suLocs, siLocs, suLocs_seq = model.sampleSslBatch(
                        batIds, model.handler.subMat, False
                    )

                    batch_data = {
                        'feed_dict': {
                            model.uids: uLocs,
                            model.iids: iLocs,
                            model.sequence: sequence,
                            model.mask: mask,
                            model.is_train: True,
                            model.uLocs_seq: uLocs_seq,
                            model.keepRate: args.keepRate
                        },
                        'step_info': (i, s, steps, len(sample_num_list))
                    }

                    # Add SSL data
                    for k in range(args.graphNum):
                        batch_data['feed_dict'][model.suids[k]] = suLocs[k]
                        batch_data['feed_dict'][model.siids[k]] = siLocs[k]
                        batch_data['feed_dict'][model.suLocs_seq[k]] = suLocs_seq[k]

                    batch_queue.put(batch_data, timeout=30)
                except queue.Full:
                    print(" Data pipeline congestion - GPU processing slower than data loading")
                    batch_queue.put(batch_data, timeout=60)  # Wait longer
                except Exception as e:
                    print(f" Data loading error: {e}")
                    break

        # Signal completion
        batch_queue.put(None)

    # Start background data loading
    data_thread = threading.Thread(target=data_loader_worker, daemon=True)
    data_thread.start()

    # GPU processing loop
    processed_batches = 0

    while True:
        try:
            batch_data = batch_queue.get(timeout=60)
            if batch_data is None:  # Completion signal
                break

            feed_dict = batch_data['feed_dict']
            i, s, steps, sample_num_list_len = batch_data['step_info']

            # Determine target operations based on hard negative usage
            if args.use_hard_neg:
                target = [model.optimizer, model.preLoss, model.regLoss, model.loss,
                         model.contrastive_loss, model.posPred, model.negPred, model.preds_one]
            else:
                target = [model.optimizer, model.preLoss, model.regLoss, model.loss,
                         model.posPred, model.negPred, model.preds_one]

            # GPU execution
            res = sess.run(target, feed_dict=feed_dict)

            if args.use_hard_neg:
                preLoss, regLoss, loss, contrastiveLoss, pos, neg, pone = res[1:]
                epochContrastiveLoss += contrastiveLoss
                if processed_batches % 10 == 0:  # Less frequent logging for efficiency
                    log('Step %d/%d: preloss = %.2f, REGLoss = %.2f, ConLoss = %.4f' %
                        (i+s*steps, steps*sample_num_list_len, preLoss, regLoss, contrastiveLoss),
                        save=False, oneline=True)
            else:
                preLoss, regLoss, loss, pos, neg, pone = res[1:]
                if processed_batches % 10 == 0:
                    log('Step %d/%d: preloss = %.2f, REGLoss = %.2f' %
                        (i+s*steps, steps*sample_num_list_len, preLoss, regLoss),
                        save=False, oneline=True)

            epochLoss += loss
            epochPreLoss += preLoss
            processed_batches += 1

        except queue.Empty:
            print(" GPU waiting for data - pipeline bottleneck detected")
            continue
        except Exception as e:
            print(f" GPU processing error: {e}")
            break

    # Wait for data loading thread to complete
    data_thread.join(timeout=10)

    ret = dict()
    ret['Loss'] = epochLoss / processed_batches if processed_batches > 0 else 0
    ret['preLoss'] = epochPreLoss / processed_batches if processed_batches > 0 else 0
    if args.use_hard_neg:
        ret['contrastiveLoss'] = epochContrastiveLoss / processed_batches if processed_batches > 0 else 0

    return ret

def run_parallel_grid_search(k_values, lambda_values, epochs, dataset_name):
    """
    GPU-optimized parallel grid search - Sequential experiments with pipeline optimization

    Note: True parallelization of TensorFlow GPU experiments is limited by:
    1. GPU memory constraints (each experiment needs ~95% of GPU memory)
    2. TensorFlow session conflicts with concurrent GPU access
    3. Google Colab single-GPU environment

    Instead, we optimize:
    - Pipeline parallelization within each experiment
    - Data loading parallelization
    - GPU utilization optimization
    - Batch processing efficiency
    """

    print(f"\n GPU-OPTIMIZED GRID SEARCH on {dataset_name.upper()}")
    print("🔧 Optimization Strategy:")
    print("   • Sequential experiments (GPU memory constraint)")
    print("   • Pipeline parallelization within experiments")
    print("   • Background data loading threads")
    print("   • XLA JIT compilation + mixed precision")
    print("   • Async GPU I/O + optimized memory allocation")
    print("   • CPU parallelization for data processing")

    total_experiments = len(k_values) * len(lambda_values)
    print(f"Total combinations: {len(k_values)} K × {len(lambda_values)} λ = {total_experiments}")

    all_results = []
    experiment_num = 0

    # GPU warming: Pre-compile TensorFlow operations
    configure_dataset(dataset_name, k_values[0], lambda_values[0])

    # Sequential execution with maximum GPU optimization per experiment
    for k_value in k_values:
        for lambda_value in lambda_values:
            experiment_num += 1

            # Ensure a clean slate for TensorFlow before each experiment run
            tf.compat.v1.reset_default_graph()
            gc.collect() # Force garbage collection

            print(f"\n{'='*20} EXPERIMENT {experiment_num}/{total_experiments} {'='*20}")
            print(f"Configuration: K={k_value}, λ={lambda_value}")

            try:
                # Run GPU-optimized experiment
                result = run_single_experiment(
                    k_value=k_value,
                    lambda_value=lambda_value,
                    epochs=epochs,
                    experiment_num=experiment_num,
                    total_experiments=total_experiments
                )

                all_results.append(result)

                # Memory cleanup between experiments (already done at start of loop, but extra gc.collect here is fine)
                gc.collect()

                # Check if the experiment actually completed successfully before trying to access detailed results
                if result and result.get("status") == "Completed":
                    print(f" Experiment {experiment_num}/{total_experiments} completed")
                    # Safely access keys, providing defaults if they might be missing
                    print(f"   Best NDCG: {result.get('best_ndcg', 0.0):.4f}")
                    print(f"   GPU Efficiency: {result.get('gpu_efficiency_batches_per_min', 0):.1f} batches/min")
                elif result:
                    print(f" Experiment {experiment_num}/{total_experiments} finished with status: {result.get('status', 'Unknown')}")
                    if result.get('error_message'):
                        print(f"   Error: {result.get('error_message')}")
                else:
                    print(f" Experiment {experiment_num}/{total_experiments} failed to return a result object.")

            except Exception as e:
                print(f" Experiment {experiment_num} failed: {e}")
                # Continue with next experiment
                continue

    return all_results

# Main execution
if GRID_SEARCH_ENABLED:
    print(f"\n🔬 Starting GPU-Optimized Grid Search on {DATASET.upper()}")

    # Use the optimized parallel grid search
    grid_search_results = run_parallel_grid_search(
        k_values=HARD_NEG_SAMPLES_K,
        lambda_values=CONTRASTIVE_WEIGHTS,
        epochs=GRID_SEARCH_EPOCHS,
        dataset_name=DATASET
    )

    print(f"\n💾 Grid search completed! {len(grid_search_results)} experiments finished.")

    # Save complete grid search summary
    save_grid_search_summary_to_drive(grid_search_results, DATASET)

    # Final grid search analysis with GPU efficiency metrics
    print("\n" + "="*80)
    print(" GPU-OPTIMIZED GRID SEARCH RESULTS ANALYSIS")
    print("="*80)

    if grid_search_results:
        # Find best configuration
        best_result = max(grid_search_results, key=lambda x: x['best_ndcg'])

        print(f" Best Configuration:")
        print(f"   K={best_result['k_value']}, λ={best_result['lambda_value']}")
        print(f"   Experiment Type: {best_result['experiment_type']}")
        print(f"   Best NDCG: {best_result['best_ndcg']:.4f}")
        print(f"   Best HR: {best_result['best_hr']:.4f}")
        print(f"   Best Epoch: {best_result['best_epoch']}")
        print(f"   Duration: {best_result['duration_minutes']:.1f} minutes")
        print(f"   GPU Efficiency: {best_result.get('gpu_efficiency_batches_per_min', 0):.1f} batches/min")

        # GPU efficiency analysis
        avg_gpu_efficiency = np.mean([r.get('gpu_efficiency_batches_per_min', 0) for r in grid_search_results])
        total_duration = sum(r['duration_minutes'] for r in grid_search_results)

        # Results table
        print(f"\n All Results Summary (sorted by NDCG):")
        print("="*100)
        print(f"{'K':<3} {'λ':<8} {'Type':<25} {'Best HR':<8} {'Best NDCG':<10} {'Epoch':<6} {'GPU Eff':<8}")
        print("-"*100)

        for result in sorted(grid_search_results, key=lambda x: x['best_ndcg'], reverse=True):
            exp_type_short = result['experiment_type'][:23] + ".." if len(result['experiment_type']) > 25 else result['experiment_type']
            gpu_eff = result.get('gpu_efficiency_batches_per_min', 0)
            print(f"{result['k_value']:<3} {result['lambda_value']:<8.3f} {exp_type_short:<25} {result['best_hr']:<8.4f} {result['best_ndcg']:<10.4f} {result['best_epoch']:<6} {gpu_eff:<8.1f}")

        # Analysis insights
        print(f"\n Analysis Insights:")

        # Best K analysis
        k_performance = {}
        for k in HARD_NEG_SAMPLES_K:
            k_results = [r for r in grid_search_results if r['k_value'] == k]
            if k_results:
                avg_ndcg = sum(r['best_ndcg'] for r in k_results) / len(k_results)
                k_performance[k] = avg_ndcg

        if k_performance:
            best_k = max(k_performance, key=k_performance.get)
            print(f"   • Best K value overall: {best_k} (avg NDCG: {k_performance[best_k]:.4f})")

        # Best λ analysis
        lambda_performance = {}
        for lam in CONTRASTIVE_WEIGHTS:
            lam_results = [r for r in grid_search_results if r['lambda_value'] == lam]
            if lam_results:
                avg_ndcg = sum(r['best_ndcg'] for r in lam_results) / len(lam_results)
                lambda_performance[lam] = avg_ndcg

        if lambda_performance:
            best_lambda = max(lambda_performance, key=lambda_performance.get)
            print(f"   • Best λ value overall: {best_lambda} (avg NDCG: {lambda_performance[best_lambda]:.4f})")

        # Edge case analysis
        baseline_result = next((r for r in grid_search_results if r['k_value'] == 0 and r['lambda_value'] == 0), None)
        if baseline_result:
            print(f"   • Pure SelfGNN (K=0, λ=0): NDCG={baseline_result['best_ndcg']:.4f}")
            improvement = ((best_result['best_ndcg'] - baseline_result['best_ndcg']) / baseline_result['best_ndcg'] * 100)
            print(f"   • Best configuration improvement over baseline: +{improvement:.2f}%")

        # Component analysis
        hard_neg_only = [r for r in grid_search_results if r['k_value'] > 0 and r['lambda_value'] == 0]
        contrastive_only = [r for r in grid_search_results if r['k_value'] == 0 and r['lambda_value'] > 0]

        if hard_neg_only:
            best_hard_neg_only = max(hard_neg_only, key=lambda x: x['best_ndcg'])
            print(f"   • Best hard negatives only (λ=0): K={best_hard_neg_only['k_value']}, NDCG={best_hard_neg_only['best_ndcg']:.4f}")

        if contrastive_only:
            best_contrastive_only = max(contrastive_only, key=lambda x: x['best_ndcg'])
            print(f"   • Best contrastive only (K=0): λ={best_contrastive_only['lambda_value']}, NDCG={best_contrastive_only['best_ndcg']:.4f}")

        print(f"\n All results saved to Google Drive at: {DRIVE_RESULTS_PATH}")
    else:
        print(" No successful experiments completed")

else:
    # Single experiment mode
    print(f"\n Running Single Experiment on {DATASET.upper()}")

    result = run_single_experiment(
        k_value=SINGLE_K,
        lambda_value=SINGLE_LAMBDA,
        epochs=SINGLE_EPOCHS
    )

    print(f"\n Single Experiment Summary:")
    print(f"  • Best NDCG@10: {result['best_ndcg']:.4f} (Epoch {result['best_epoch']})")
    print(f"  • Best HR@10: {result['best_hr']:.4f}")
    print(f"  • Final NDCG@10: {result['final_ndcg']:.4f}")
    print(f"  • Final HR@10: {result['final_hr']:.4f}")
    print(f"  • Duration: {result['duration_minutes']:.1f} minutes")
    print(f" Result saved to Google Drive at: {DRIVE_RESULTS_PATH}")

In [ ]:
import os
os.system('kill -9 -1')